# CO₂–NH₃ Cascade Refrigeration Design for a -20 °C Freezer (LLM-Generated Solution)

### Problem Summary
To reduce the use of synthetic refrigerants, we are designing a natural refrigerant-based cascade freezer system for an industrial application. The system uses carbon dioxide (CO₂) on the cold side and ammonia (NH₃) on the hot side.

- The freezer operates at –20 °C, so the CO₂ evaporator must reach this temperature.
- The CO₂ condenser and NH₃ evaporator are thermally coupled in a cascade heat exchanger.
- This notebook attempts to identify optimum intermediate temperatures, assign a cooling capacity, and pick a realistic application.

Assumptions
- The CO₂ cycle evaporates at –20 °C and condenses at an arbitrary 10 °C
- The NH₃ cycle evaporates at 5 °C and condenses at 35 °C
- Isentropic compression and expansion
- Idealized cycles (no pressure drops or heat exchanger irreversibilities)
- Cooling capacity = 100 kW (typical for a medium cold storage unit)
- Operating location: East Bay, California
- Typical summer ambient temperature: 30 °C
- Property data via CoolProp


In [1]:
import CoolProp.CoolProp as CP
import numpy as np

# Refrigerants
refrigerant_CO2 = 'CO2'
refrigerant_NH3 = 'Ammonia'

# Operating temperatures (K)
T_evap_CO2 = 253.15  # -20 °C
T_cond_CO2 = 283.15  # 10 °C (arbitrarily chosen)

T_evap_NH3 = 278.15  # 5 °C (should match CO2 condenser)
T_cond_NH3 = 308.15  # 35 °C

# Pressures
P_evap_CO2 = CP.PropsSI('P', 'T', T_evap_CO2, 'Q', 1, refrigerant_CO2)
P_cond_CO2 = CP.PropsSI('P', 'T', T_cond_CO2, 'Q', 0, refrigerant_CO2)

P_evap_NH3 = CP.PropsSI('P', 'T', T_evap_NH3, 'Q', 1, refrigerant_NH3)
P_cond_NH3 = CP.PropsSI('P', 'T', T_cond_NH3, 'Q', 0, refrigerant_NH3)

# Enthalpies (CO2 cycle)
h1_CO2 = CP.PropsSI('H', 'T', T_evap_CO2, 'Q', 1, refrigerant_CO2)  # saturated vapor
h4_CO2 = CP.PropsSI('H', 'T', T_evap_CO2, 'Q', 0, refrigerant_CO2)  # saturated liquid

# Enthalpies (NH3 cycle)
h1_NH3 = CP.PropsSI('H', 'T', T_evap_NH3, 'Q', 1, refrigerant_NH3)
h4_NH3 = CP.PropsSI('H', 'T', T_evap_NH3, 'Q', 0, refrigerant_NH3)

# Cooling capacity (W)
Qe = 100_000

# Mass flow rates
m_CO2 = Qe / (h1_CO2 - h4_CO2)

# Compressor work (isentropic)
s1_CO2 = CP.PropsSI('S', 'T', T_evap_CO2, 'Q', 1, refrigerant_CO2)
h2s_CO2 = CP.PropsSI('H', 'P', P_cond_CO2, 'S', s1_CO2, refrigerant_CO2)
W_CO2 = m_CO2 * (h2s_CO2 - h1_CO2)

s1_NH3 = CP.PropsSI('S', 'T', T_evap_NH3, 'Q', 1, refrigerant_NH3)
h2s_NH3 = CP.PropsSI('H', 'P', P_cond_NH3, 'S', s1_NH3, refrigerant_NH3)
# Assume same m_dot for ammonia as CO2 (simplifying assumption)
W_NH3 = m_CO2 * (h2s_NH3 - h1_NH3)

# Total work input
W_total = W_CO2 + W_NH3

# COP and Second-law efficiency (assumed Carnot COPs for ideal)
COP_actual = Qe / W_total
COP_Carnot_CO2 = T_evap_CO2 / (T_cond_CO2 - T_evap_CO2)
eta_II = COP_actual / COP_Carnot_CO2

# Application
application = "Frozen food warehouse in East Bay, CA"

# Output results
print(f"Application: {application}")
print(f"Cooling capacity: {Qe/1000:.1f} kW")
print(f"CO₂ condenser temperature: {T_cond_CO2 - 273.15:.1f} °C")
print(f"NH₃ evaporator temperature: {T_evap_NH3 - 273.15:.1f} °C")
print(f"Total compressor work: {W_total/1000:.1f} kW")
print(f"System COP: {COP_actual:.2f}")
print(f"Second Law Efficiency (η_II): {eta_II:.2f}")


Application: Frozen food warehouse in East Bay, CA
Cooling capacity: 100.0 kW
CO₂ condenser temperature: 10.0 °C
NH₃ evaporator temperature: 5.0 °C
Total compressor work: 59.8 kW
System COP: 1.67
Second Law Efficiency (η_II): 0.20
